## Usando arquivo .env para controlar variaveis de ambiente
Para evitar exposição da chave `OPENAI_API_KEY` optei por utilizar arquivo `.env` com a informação da chave.

Para seguir o mesmo método basta criar um arquivo `.env` no mesmo diretório do arquivo `dataset_prep.ipynb`.
A importação da chave será feita através da célula abaixo que faz a instalação de um biblioteca para carregar
os valores do arquivo `.env`.

In [ ]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

### Importando Dataset

Neste vídeo, começamos um novo projeto no VS Code, criando um notebook Jupyter chamado "Dataset prep.ipynb". Importamos as bibliotecas necessárias e salvamos um arquivo com as versões das libs usando o comando "pip freeze". Utilizamos a biblioteca "datasets" para carregar um dataset diretamente do Hugging Face, removemos colunas desnecessárias e reduzimos o tamanho do dataset para 567 linhas, visando reduzir custos de treinamento. Essa abordagem pode impactar a qualidade do fine tuning, mas é útil para fins de aprendizado.

In [3]:
import json
from datasets import load_dataset

In [4]:
datasets = load_dataset("hate-speech-portuguese/hate_speech_portuguese", split='train[:10%]')

In [5]:
print(datasets)

Dataset({
    features: ['text', 'label', 'hatespeech_G1', 'annotator_G1', 'hatespeech_G2', 'annotator_G2', 'hatespeech_G3', 'annotator_G3'],
    num_rows: 567
})


In [6]:
datasets = datasets.remove_columns(['hatespeech_G1', 'annotator_G1', 'hatespeech_G2', 'annotator_G2', 'hatespeech_G3', 'annotator_G3'])
datasets

Dataset({
    features: ['text', 'label'],
    num_rows: 567
})

### Train Test Split

Neste trecho, explicamos como dividir um conjunto de dados em treino e teste para realizar o ajuste fino de um modelo de machine learning. Destacamos a importância de ter um conjunto de teste para avaliar o desempenho do modelo. Também abordamos a complexidade de construir manualmente um conjunto de dados para treinamento, ressaltando a necessidade de cuidado ao utilizar modelos pré-treinados para essa tarefa. A divisão do conjunto de dados em treino e teste é essencial para validar o modelo.

In [7]:
datasets = datasets.train_test_split(test_size=0.2)

In [8]:
datasets['train']['text'][0]

'A mídia DE NOTÍCIAS FALSAS desconhece a verdade deliberadamente. Grande perigo ao país. O @nytimes falido virou pia _ https://t.co/ssibMARmkc'

### Manipulando os Dados

Neste trecho, é abordado o uso da função map para manipular um dataset. É criada uma função para remover caracteres específicos, como "\n", e outra para alterar os rótulos das categorias. São feitas correções para garantir a execução correta das funções. Ao final, o dataset é modificado e exibido para verificar as alterações realizadas.

In [9]:
def removeN(example):
    example["text"] = example["text"].replace("\n", " ")
    return example

In [10]:
datasets = datasets.map(removeN)

Map:   0%|          | 0/453 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

In [11]:
datasets['train']['text'][0]

'A mídia DE NOTÍCIAS FALSAS desconhece a verdade deliberadamente. Grande perigo ao país. O @nytimes falido virou pia _ https://t.co/ssibMARmkc'

### label 0 -> No Hate Speech
### label 1 -> Hate Speech

In [12]:
def labelChange(example):
    example["label_text"] = "No Hate Speech" if  example["label"] == 0 else "Hate Speech"
    return example

In [13]:
datasets = datasets.map(labelChange)
datasets

Map:   0%|          | 0/453 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 453
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 114
    })
})

In [14]:
datasets = datasets.remove_columns(['label'])
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label_text'],
        num_rows: 453
    })
    test: Dataset({
        features: ['text', 'label_text'],
        num_rows: 114
    })
})

In [15]:
datasets['train'][0]

{'text': 'A mídia DE NOTÍCIAS FALSAS desconhece a verdade deliberadamente. Grande perigo ao país. O @nytimes falido virou pia _ https://t.co/ssibMARmkc',
 'label_text': 'No Hate Speech'}

### Finalizando Dataset

Estamos quase finalizando nosso dataset no formato desejado para a OpenAI. Vamos estruturar as mensagens com os papéis de sistema, usuário e assistente, garantindo que o conteúdo do usuário seja o texto e a resposta do assistente seja a classificação de discurso de ódio. Criaremos uma função em Python para transformar nosso dataset em um arquivo JSONL, que é o que a OpenAI espera. Também discutimos a importância do fine-tuning e os custos envolvidos nesse processo.

In [16]:
# CONSTRUÇÃO DO OBJETO PARA OPEN AI
def dataset_to_json(datasets, file_name):
    with open(file_name, 'w', encoding="utf-8") as f:
        for example in datasets:
            json_obj = {
                "messages": [
                    {"role": "system", "content": "Seu trabalho é classificar os comentários do usuário em Hate Speech ou No Hate Speech."},
                    {"role": "user", "content": example["text"]},
                    {"role": "assistant", "content": example["label_text"]}
                ]
            }
            f.write(json.dumps(json_obj, ensure_ascii=False) + '\n')

### Fine-tuning GPT OpenAI

Nesta aula, vamos finalizar a construção do Fine Tuning utilizando a API da OpenAI. Começamos importando as bibliotecas necessárias e conectando nossa chave de API. Em seguida, fazemos o upload dos arquivos de treinamento e validação, especificando seus propósitos. Após isso, iniciamos o processo de Fine Tuning com o modelo GPT-3.5 Turbo. Acompanhe o progresso no dashboard da OpenAI e, em breve, veremos os resultados do nosso trabalho.

In [ ]:
import os
from openai import OpenAI

client = OpenAI()

os.environ["OPENAI_API_KEY"] = ""

In [22]:
file_train = client.files.create(
    file=open("train.jsonl", "rb"),
    purpose="fine-tune",
)

In [23]:
file_validation = client.files.create(
    file=open("validation.jsonl", "rb"),
    purpose="fine-tune",
)

In [21]:
# Devido ao Fine-tune ter sido descontinuado para modelo davinci-002 estarei utilizando o modelo mais recente até momento de saída da aula e também recomendado pela documentação
# https://platform.openai.com/docs/guides/fine-tuning#create-a-fine-tuned-model
client.fine_tuning.jobs.create(
    training_file=file_train.id,
    validation_file=file_validation.id,
    model="gpt-4o-mini-2024-07-18"
)

FineTuningJob(id='ftjob-WJ260o5JFtFYO2Lc56LC6Xi0', created_at=1737333203, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-9Ka4MdZAwINdpOMmxZxue32b', result_files=[], seed=568076911, status='validating_files', trained_tokens=None, training_file='file-Pq8eAq7LNP69yb8nPerGxF', validation_file='file-UMJvtV1xTSft5WkTbkND1N', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None)

In [ ]:
def dataset_to_json_aws(datasets, file_name):
    with open(file_name, 'w', encoding="utf-8") as f:
        for example in datasets:
            json_obj = {
                "prompt": example["text"],
                "completion": example["label_text"]
            }
            f.write(json.dumps(json_obj, ensure_ascii=False) + '\n')

In [ ]:
dataset_to_json_aws(datasets["train"], "train_aws.jsonl")

In [ ]:
dataset_to_json_aws(datasets["test"], "validation_aws.jsonl")